In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Attention
from tensorflow.keras.models import Model
import re
import numpy as np
from tensorflow.keras.models import load_model

import pandas as pd
import glob

In [4]:
import pandas as pd
import re
import glob

file_pattern = f"data/conve*.txt"
def extract_number(file_name):
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else None

matching_files = glob.glob(file_pattern)
filtered_files = [file for file in matching_files if 0 <= extract_number(file) <= 50]

print(len(filtered_files))
print(len(matching_files))
input_texts = []
output_texts = []
df = pd.DataFrame()
for file_path in matching_files:
    with open(file_path, "r") as file:
        content = file.read()

    examples = content.split("assistant: ")

    inputs = []
    outputs = []

    i = 1
    for example in examples[1:]:
        for _ in range(100):
            input_match = re.search(f"Input{i}: (.+?)\n", example)
            output_match = re.search(f"Output{i}: (.+?)\n", example)

            if input_match and output_match:
                outputs_value = output_match.group(1)
                inputs_value = re.sub(r'Output.*', r'', input_match.group(1))
                inputs.append(inputs_value)
                outputs.append(outputs_value)
                output_texts.append(outputs_value)
                input_texts.append(inputs_value)
            i += 1  
    newDf = pd.DataFrame({"input": inputs, "output": outputs})
    df = pd.concat([df, newDf], ignore_index=True)


50
128


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
import numpy as np

max_input_seq_len = max(len(seq) for seq in input_texts)
max_output_seq_len = max(len(seq) for seq in output_texts)

input_tokenizer = Tokenizer(filters='', char_level=False)
output_tokenizer = Tokenizer(filters='', char_level=False)

input_tokenizer.fit_on_texts(input_texts)
print(len(input_tokenizer.word_index))
output_tokenizer.fit_on_texts(output_texts)

encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_seqs = output_tokenizer.texts_to_sequences(output_texts)
decoder_target_seqs = [seq[1:] for seq in decoder_input_seqs]

encoder_input_seqs = pad_sequences(encoder_input_seqs, maxlen=max_input_seq_len, padding='post')
decoder_input_seqs = pad_sequences(decoder_input_seqs, maxlen=max_output_seq_len, padding='post')
decoder_target_seqs = pad_sequences(decoder_target_seqs, maxlen=max_output_seq_len, padding='post')

latent_dim = 256

encoder_inputs = Input(shape=(max_input_seq_len,))
encoder_embedding = Embedding(input_dim=len(input_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_output_seq_len,))
decoder_embedding = Embedding(input_dim=len(output_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

attention = Attention()([decoder_outputs, encoder_lstm])
context = Concatenate(axis=-1)([attention, decoder_outputs])

decoder_dense = Dense(len(output_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(context)




4412


In [6]:

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=10, batch_size=1)

model.save('my_model_files_0-50_dim_256.h5')


Epoch 1/10


2023-11-23 00:09:16.790418: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


996/996 [==============================] - 907s 907ms/step - loss: 3.3109 - accuracy: 0.4130
Epoch 2/10
996/996 [==============================] - 889s 892ms/step - loss: 2.0946 - accuracy: 0.5574
Epoch 3/10
996/996 [==============================] - 874s 877ms/step - loss: 1.5810 - accuracy: 0.6198
Epoch 4/10
996/996 [==============================] - 878s 881ms/step - loss: 1.2243 - accuracy: 0.6722
Epoch 5/10
996/996 [==============================] - 863s 867ms/step - loss: 0.9609 - accuracy: 0.7149
Epoch 6/10
996/996 [==============================] - 865s 868ms/step - loss: 0.7588 - accuracy: 0.7583
Epoch 7/10
996/996 [==============================] - 863s 867ms/step - loss: 0.6098 - accuracy: 0.7996
Epoch 8/10
996/996 [==============================] - 865s 869ms/step - loss: 0.4977 - accuracy: 0.8334
Epoch 9/10
996/996 [==============================] - 859s 863ms/step - loss: 0.4122 - accuracy: 0.8602
Epoch 10/10
996/996 [==============================] - 859s 862ms/step - lo

/home/ukasz/anaconda3/envs/ml_lab/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
#model4 is the best
model = load_model('my_model4.h5')

# input_sentence = "Scan a list of prices until finding the one with the highest value"
input_sentence = "Look for the highest score in a game leaderboard"
# input_sentence = "Practice speaking, writing, and listening in each language."

input_sequence = input_tokenizer.texts_to_sequences([input_sentence])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_len, padding='post')

decoded_sequence = np.zeros((1, max_output_seq_len))

for i in range(max_output_seq_len):
    predictions = model.predict([padded_input_sequence, decoded_sequence])
    predicted_token_index = np.argmax(predictions[0, i, :])
    decoded_sequence[0, i] = predicted_token_index

    if predicted_token_index == 0: 
        break

output_sentence = output_tokenizer.sequences_to_texts([decoded_sequence[0]])

print("Input sentence:", input_sentence)
print("Predicted output:", output_sentence)

2023-12-07 08:20:20.403931: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 0s 189ms/step
Input sentence: Look for the highest score in a game leaderboard
Predicted output: ['the --> list] you set your time with a new number] b -- no --> a b -- yes --> c[learn from setbacks and keep going] c --> d[end]']


In [21]:
existing_model = load_model('my_model_latent_dim_16.h5')
existing_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
existing_model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=10, batch_size=1)
existing_model.save('my_model_latent_dim_64.h5')

Epoch 1/10


2023-11-20 21:22:27.627862: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


2293/2293 [==============================] - 581s 252ms/step - loss: 3.4928 - accuracy: 0.4325
Epoch 2/10
2293/2293 [==============================] - 566s 247ms/step - loss: 3.1967 - accuracy: 0.4574
Epoch 3/10
2293/2293 [==============================] - 565s 247ms/step - loss: 2.9853 - accuracy: 0.4779
Epoch 4/10
2293/2293 [==============================] - 567s 247ms/step - loss: 2.8127 - accuracy: 0.4947
Epoch 5/10
2293/2293 [==============================] - 562s 245ms/step - loss: 2.6676 - accuracy: 0.5065
Epoch 6/10
2293/2293 [==============================] - 561s 245ms/step - loss: 2.5374 - accuracy: 0.5198
Epoch 7/10
2293/2293 [==============================] - 559s 244ms/step - loss: 2.4211 - accuracy: 0.5340
Epoch 8/10
2293/2293 [==============================] - 560s 244ms/step - loss: 2.3187 - accuracy: 0.5451
Epoch 9/10
2293/2293 [==============================] - 563s 245ms/step - loss: 2.2252 - accuracy: 0.5559
Epoch 10/10
2293/2293 [==============================] - 

In [18]:
model = load_model('my_model_latent_dim_16.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 420)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 653)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 420, 16)              70608     ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 653, 16)              123488    ['input_4[0][0]']             
                                                                                              